In [23]:
import pandas as pd
import numpy as np
import wrangle
from pydataset import data

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [2]:
# bringing in the data
tips_df = data('tips')

In [3]:
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


### 1.) Load the tips dataset.
- a.) Create a column named tip_percentage. This should be the tip amount divided by the total bill.
- b.) Create a column named price_per_person. This should be the total bill divided by the party size.
- c.) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?
- d.) Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?
- e.) Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?
- f.) Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [4]:
#part a 
tips_df['tip_percentage'] = round(tips_df['tip']/tips_df['total_bill'], 2)
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15


In [5]:
#part b 
tips_df['price_per_person'] = round(tips_df['total_bill']/tips_df['size'],2)

In [6]:
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15,6.15


tips_df['sex'].replace(['Female','Male'],[0,1],inplace=True)
tips_df['smoker'].replace(['No','Yes'],[0,1],inplace=True)
tips_df['time'].replace(['Lunch','Dinner'],[0,1],inplace=True)
tips_df['day'].replace(['Thur','Fri', 'Sat', 'Sun'],[0,1,2,3],inplace=True)

In [7]:
# part c
# my initial assumption would be time and size would be the most important features in predicting tip amount.
# for predicting the tip percentage, I would think total bill.

In [13]:
#part d 
X = tips_df[['total_bill', 'size', 'tip_percentage', 'price_per_person']]
y = tips_df['tip']

from sklearn.feature_selection import SelectKBest, f_regression

f_selector = SelectKBest(score_func=f_regression, k=3)
f_selector.fit(X, y)


SelectKBest(k=3, score_func=<function f_regression at 0x7f8c26d64550>)

In [14]:
f_selector.get_support()

array([ True,  True, False,  True])

In [15]:
f_selector.transform

<bound method SelectorMixin.transform of SelectKBest(k=3, score_func=<function f_regression at 0x7f8c26d64550>)>

In [16]:
X = tips_df[['total_bill', 'size', 'tip', 'price_per_person']]
y = tips_df['tip_percentage']

from sklearn.feature_selection import SelectKBest, f_regression

f_selector = SelectKBest(score_func=f_regression, k=3)
f_selector.fit(X, y)

SelectKBest(k=3, score_func=<function f_regression at 0x7f8c26d64550>)

In [17]:
f_selector.get_support()


array([ True, False,  True,  True])

In [18]:
f_selector.transform

<bound method SelectorMixin.transform of SelectKBest(k=3, score_func=<function f_regression at 0x7f8c26d64550>)>

### 2.) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [20]:
def select_kbest(X, y, k):
    
    f_selector = SelectKBest(f_regression, k)
    f_selector.fit(X, y)
    
    f_features = X.columns[f_selector.get_support()]
    
    return f_features



In [22]:
select_kbest(X, y, 3)

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass k=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Index(['total_bill', 'tip', 'price_per_person'], dtype='object')

### 3.) Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [33]:
def rfe(X, y, k):
    '''
    Must have sklearn linear regression imported
    '''
    lm = LinearRegression()
    rfe = RFE(estimator=lm, n_features_to_select = k)
    rfe.fit(X, y)
    
    f_features = X.columns[rfe.support_]
    
    return f_features
    
 

In [34]:
rfe(X, y, 2)

Index(['size', 'tip'], dtype='object')

### 4.) Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [35]:
### predict fertility (target)
### top 3 features
swiss_df = data('swiss')

In [36]:
swiss_df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [40]:
X = swiss_df.drop(columns = 'Fertility')
y = swiss_df['Fertility']

In [41]:
select_kbest(X, y, 3)

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass k=3 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Index(['Examination', 'Education', 'Catholic'], dtype='object')

In [42]:
rfe(X, y, 3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')